In [1]:
import ee,requests,os,sys
from IPython.display import Image
ee.Initialize(project='lcms-292214')

import geeViz.getImagesLib as gil
import geeViz.changeDetectionLib as cdl
Map = gil.Map

c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


geeViz package folder: c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\geeViz


In [4]:
# Get CONUS Loss and Loss+Gain images

output_folder = '../assets/images'
version = '2023-9'
conus_loss = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss.png')
conus_loss_gain = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss_Gain.png')
print(conus_loss)
gil.vizParamsTrue = {'min': 0.1, 'max': [0.3, 0.3, 0.3], 'bands': 'red,green,blue'}
sa = ee.FeatureCollection("projects/lcms-tcc-shared/assets/CONUS/Ancillary/Conus_NLCD_Boundary")

crs = gil.common_projections['NLCD_CONUS']['crs']
transform = gil.common_projections['NLCD_CONUS']['transform']
# transform[0] = 300
# transform[-2] = 300
print(transform)

def getMostRecentChange(c, code):
    def wrapper(img):
        yr = ee.Date(img.get("system:time_start")).get("year")
        out = ee.Image(yr).int16()
        out = out.where(img.neq(code),0).setDefaultProjection(crs,transform)#.selfMask()
        return (
            ee.Image(out
            .rename(["year"])
            .copyProperties(img, ["system:time_start"])).reduceResolution(ee.Reducer.firstNonNull(),True,256).selfMask()#.reproject(crs,transform)#.selfMask()
        )

    return c.map(wrapper)
def downloadImage(url,filename):
    img_data = requests.get(url).content
    with open(filename, 'wb') as handler:
        handler.write(img_data)

comp = ee.ImageCollection("projects/lcms-tcc-shared/assets/CONUS/Composites/Composite-Collection-yesL7")\
.filter(ee.Filter.calendarRange(2020,2023,'year')).mean()#.resample('bicubic').reproject(crs,transform)

comp = comp.visualize(**gil.vizParamsTrue10k)#.setDefaultProjection(crs,transform)#.reduceResolution(ee.Reducer.mean(),True).reproject(crs,transform)

lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9").filter("study_area=='CONUS'").select([0])

fast_loss = getMostRecentChange(lcms,3).mean().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
slow_loss = getMostRecentChange(lcms,2).mean().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
gain = getMostRecentChange(lcms,4).mean().visualize(**{'min':1985,'max':2023,'palette':cdl.gainYearPalette})

loss = comp
loss = loss.where(fast_loss.neq(0),fast_loss)
loss = loss.where(slow_loss.neq(0),slow_loss)

loss_gain = comp

loss_gain = loss_gain.where(fast_loss.neq(0),fast_loss)
loss_gain = loss_gain.where(slow_loss.neq(0),slow_loss)
loss_gain = loss_gain.where(gain.neq(0),gain)
Map.clearMap()
Map.addLayer(loss,{},'Loss')
Map.addLayer(loss_gain,{},'Loss+Gain')
Map.view()

params = {
  'region': sa.geometry().bounds(500,crs),
  'dimensions': 1000,
  'crs': crs,
  'transform':transform,
  'framesPerSecond': 5
}

# Get a URL that will generate a GIF.
# url = comps.getVideoThumbURL(params)
loss_url = loss.getThumbURL(params)
loss_gain_url = loss_gain.getThumbURL(params)

downloadImage(loss_url,conus_loss)
downloadImage(loss_gain_url,conus_loss_gain)
# print(url)
# # Display the NDVI animation.
# Image(url=url)


../assets/images\CONUS_v2023-9_Change_Loss.png
[30, 0, -2361915.0, 0, -30, 3177735.0]
Adding layer: Loss
Adding layer: Loss+Gain
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd z:\Projects\06_LCMS_4_NFS\Scripts\landscape-change-data-explorer\src\setup
geeView URL: http://localhost:8001/geeView/?projectID=lcms-292214&accessToken=ya29.a0Ad52N3_ZqrcIswn-acajr28kftj2QeEGCwnYHrXr5bPxQ8nNBWiwRrj3MoDZPbvGkpW4WpIxxKhncV37BH_OyPiQggNEfHCoIC1DEIHwbXrCxcV8OEoP5hKiSoU98b6W1JlvD11G3dj_W_NF8oF4FfxnaA0IJiM9Dqu63GgaCgYKAToSARESFQHGX2MiF633u7IQh0Kk7xrih3N6jQ0174


{'type': 'Image', 'bands': [{'id': 'vis-red', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'vis-green', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'vis-blue', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:annotations': None}}
39
